In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# Data and package import

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
# read raw data
# na_values: defines values that should be viewed as nan

RawEGEDA = pd.read_excel('./data/October_2022/00APEC.xlsx',\
                         sheet_name = None, # 'None' reads in all sheets\
                         na_values = ['x', 'X', '']) # I don't think there's any x's or X's in the EGEDA xlsx file, but leaving as is (shouldn't make a difference)

## Check the shapes of all dataframes (確認每一個資料框的列欄數是否相同)

In [ ]:
# Save the economy name (key) in RawEGEDA. We will use it in the for-loop. (將 egeda 的經濟體標示法存起來，後面在對 egeda 資料做迴圈的時候會用到)
economies = RawEGEDA.keys()

# economies

In [ ]:
# Use for-loop to record the shape of all dataframes (透過迴圈將每個資料框的 shape 存進 shape_of_data 的 list 當中)
shape_of_data = []

for i in economies:
    dimension = RawEGEDA[i].shape
    shape_of_data.append(dimension)
    
# shape_of_data

### Check the numbers of row (檢查列)

In [ ]:
# save the numbers of row of all dataframe (把每個資料框 shape 的列數均存在一個 list 裡面)
number_of_rows = []
for i in shape_of_data:
    number_of_rows.append(i[0])

# number_of_rows

In [ ]:
# Use the number of row of first dataframe as reference, and check if the others are the same as it via for-loop. 
# Save the result in a list. (以列表中第一個數值為標準 (第一個資料框的列數)，透過迴圈檢查美個資料框的列數是否與之相同，並將結果生成為布林值存成一個列表)

boolean_list = []
for i in number_of_rows:
    boolean_list.append(i == number_of_rows[0])

if all(boolean_list) == True: # all() returns True only if all elements in the list are True (all() 只有在括號中的列表全部都是 True 時才會回傳 True)
    print("the number of rows are all the same!")
else:
    print("somthing worng")

### Check the numbers of columns (檢查欄)
- We found some inconsistent numbers of columns.
- It resulted from the fact that some economies updated their 2021 data. (不一致問題來自有些經濟體更新了 2021 的資料，有些則沒有)

In [ ]:
# save the numbers of columns of all dataframe (把每個資料框 shape 的欄數均存在一個 list 裡面)
number_of_columns = []
for i in shape_of_data:
    number_of_columns.append(i[1])
    
# number_of_columns

In [ ]:
# Use the number of column of first dataframe as reference, and check if the others are the same as it via for-loop. 
# Save the result in a list.(以列表中第一個數值為標準 (第一個資料框的欄數)，透過迴圈檢查美個資料框的欄數是否與之相同，並將結果生成為布林值存成一個列表)
# all() 只有在括號中的列表全部都是 True 時才會回傳 True
boolean_list = []
for i in number_of_rows:
    boolean_list.append(i == number_of_columns[0])

if all(boolean_list) == True: # all() returns True only if all elements in the list are True (all() 只有在括號中的列表全部都是 True 時才會回傳 True)
    print("the number of columns are all the same!")
else:
    print("something worng")

## Name the first two columns which are currently blank (幫 egeda 原始資料缺乏欄位名稱的前兩欄命名)

In [ ]:
for i in economies:
    RawEGEDA[i].rename(columns = {'Unnamed: 0': 'fuels', 'Unnamed: 1': 'sectors'}, inplace = True)

## Prepare lists for dataframe merging process later

### Year
- Some regions have 2021 data but it is incomplete, and so we only want data to 2020
- Let the last year of historical data be 2020.

In [ ]:
years = list(range(1980, 2020+1, 1))

In [ ]:
years[-1]

### Economy
- In raw data, the item numbers are: numbers+economyname (egeda 中以「編號+經濟體代碼」的方式標示)
- We would like: numbers+_+economyname (我們想要的則是「編號+_+經濟體代碼」)
- Load the reference table that we prepared in advance. (我們透過一個另外準備的 csv 檔案，裡面則為兩種標示法的對照，並以字典方式讀入)

In [ ]:
economy_dict = pd.read_csv('./data/economy_dict.csv', header = None, index_col = 0).squeeze('columns').to_dict()
# economy_dict

# From dict to dataframe and From wide-df to long df (把字典的資料框合併在一起同時將資料轉成長表格)
- In the for-loop, we melted the wide-df into long-df and saved them in a list one after another.  (對象為字典中的每一個資料框)
- Then we concat all df in the list into a large one.

## Single economy for example (範例：單一經濟體)

## Combine all dataframes into single large one: all economies for-loop (迴圈跑遍所有經濟體，並將多個經濟體的資料合併為一個巨大資料框)

In [ ]:
df_list = []

for i in economies:
    temp_ebt_df_economy_wide = RawEGEDA[i]
    temp_ebt_df_economy_long = pd.melt(temp_ebt_df_economy_wide, id_vars = ['fuels', 'sectors'], value_vars = years, var_name = 'year', value_name = 'value')
    temp_ebt_df_economy_long['economy'] = economy_dict[i] # add a new col called 'economy', which is in the format that we preferred (e.g., 01_AUS). (這邊新增一欄，並使用先前字典裏面的經濟體對應關係去對應)
    temp_ebt_df_economy_long = temp_ebt_df_economy_long.set_index(['economy', 'year']) # Use economy and year as index (將經濟體與年份設定為索引值)
    df_list.append(temp_ebt_df_economy_long) # Save all df into the list (把各經濟體的資料框依序存在清單中)

In [ ]:
df = pd.concat(df_list) # vertical combine (將清單中的經濟體做垂直合併)

In [ ]:
df.info()

## Standarlization (標準化)
- standarlize col names and variable names
- blank, special symbols are not allowed (空格、特殊符號都是不被允許的)
- use underscore if necessary. (如果真有必要使用，使用 _)
- notice that we replace '&' with 'and'.

### Col name (標準化欄位名稱)
- In this case, the col names do not need to be changed.

#### blank (空格換底線)

#### uppercase (大寫換小寫)

### variable names (標準化數值)

#### Remove multiple blank (移除多餘的「連續空格」)
- \s indicates one or multiple (\s 在正則表達式中，代表一個或多個連續空格)

In [ ]:
# And remove multiple spaces from variables
df['fuels'] = df['fuels'].replace('\s+', ' ', regex = True)
df['sectors'] = df['sectors'].replace('\s+', ' ', regex = True)

#### Convert to lowercase (全部變小寫)

In [ ]:
df['fuels'] = df['fuels'].str.lower()
df['sectors'] = df['sectors'].str.lower()

#### Underscore rocks (細節修正：底線 rocks)

#####  Replace all (正式排除對原始資料框進行處理)

In [ ]:
# fuels_code

df['fuels'] = df['fuels'].str.replace(' ', '_', regex = False)\
                                       .str.replace('.', '_', regex = False)\
                                       .str.replace('/', '_', regex = False)\
                                       .str.replace('(', '', regex = False)\
                                       .str.replace(')', '', regex = False)\
                                       .str.replace('-', '', regex = False)\
                                       .str.replace(',', '', regex = False)\
                                       .str.replace('&', 'and', regex = False)\
                                       .str.replace('___', '_', regex = False)\
                                       .str.replace('__', '_', regex = False)\
                                       .str.replace(':', '', regex = False)\
                                       .str.replace('liqour', 'liquor', regex = False)\
                                       .str.rstrip('_')

# sectors_code

df['sectors'] = df['sectors'].str.replace(' ', '_', regex = False)\
                                 .str.replace('.', '_', regex = False)\
                                 .str.replace('/', '_', regex = False)\
                                 .str.replace('(', '', regex = False)\
                                 .str.replace(')', '', regex = False)\
                                 .str.replace('-', '', regex = False)\
                                 .str.replace(',', '', regex = False)\
                                 .str.replace('&', 'and', regex = False)\
                                 .str.replace('___', '_', regex = False)\
                                 .str.replace('__', '_', regex = False)\
                                 .str.replace(':', '', regex = False)\
                                 .str.rstrip('_')

#### Transfer item_number into two digits (將 item_number 換成兩位數)

In [ ]:
df['fuels'] = df['fuels'].apply(lambda x: re.sub(r'\d+', lambda y: y.group(0).zfill(2), x))
df['sectors'] = df['sectors'].apply(lambda x: re.sub(r'\d+', lambda y: y.group(0).zfill(2), x))

In [ ]:
# df[((df['fuels'] == '03_peat') | (df['fuels'] == '04_peat_products')) & ((df['value'] != 0))]

## Comparison between current layout and your self-defined layout (比較 WU_layout 跟迄今的資整結果)
- The above process convert the EGEDA raw data into a dataframe in a well-accepted format (df).
- However, the ultimate purpose of this file is to prepare a modeller-friendly historical data and layout for modeller to use and fill in their model result. 
- We probably need to make some new definition or add new rows to the energy balance table (EBT). Let's call it sd_layout here.
    - We created a new datafolder in data folders and put the sd_layout of fuel and sector in it.
        - .\data\fuel_list.xlsx
        - .\data\sector_list.xlsx
- The features of sd_layout
    - multiple cols for sector(5) and fuels(2) (df only have one sector col and one fuel col)
    - if we use the last_col for sector and fuel of the sd_layout, the layout would be similar to current df.
    - the format generally follow the above rules
    - **item number should not be the same as the current df** because we probably do aggregation or add new fuels/ sectors already.
- So the idea here would be  
    1. (**Do it in python**) export the current df to excel file  
        - .\temp\clean_egeda_fuel_name.xlsx
        - .\temp\clean_egeda_sector_name.xlsx
    2. create new files 
        - .\data\reference_table_fuel.xlsx 
        - .\data\reference_table_sector.xlsx
    3. copy the clean_egeda cols and the sd_layout, and then paste them to the excel files that you just created  
    4. align the name of fuels and sector, and mark the differences in other cols.
    5. (**Do it in python**) Do final check for the remaining inconsistent parts in python (**notice that we do not revise in python but in excel**)
        - read the excel files, drop the item numbers, and do the comparison in this file.  
        - You may find the cases like
            - exactly the same
            - minor difference
            - exist in current df but not sd_layout (aggregation)
            - exist in sd_layout but not current df (new-defined or minus)
    4. fix the minor difference in excel and save it as new excel file (e.g., reference_table_fuel_revised.xlsx).
    5. import the file (except for item number, there should not be minor difference anymore)
- (Important) Paste the revised fule and sector cols to these files which record almost every edition of the fule and sector name. Remember to revise the multiple cols and make it the same data format, we will use this file later.
    - EBT_column_fuels.xlsx
    - EBT_row_sectors.xlsx

### Export the fuel and sector columns from df (輸出迄今為止的 sector 與 fuel 進行觀察)


In [ ]:
unique_fuels = df['fuels'].unique()
unique_sectors = df['sectors'].unique()

In [ ]:
unique_fuels_df = pd.DataFrame({'clean_egeda_fuel_name': unique_fuels})
# unique_fuels_df

In [ ]:
unique_sectors_df = pd.DataFrame({'clean_egeda_sector_name': unique_sectors})
# unique_sectors_df

In [ ]:
import os
folder_path = './temp'
os.makedirs(folder_path, exist_ok=True)

result_path = os.path.join(folder_path, 'clean_egeda_fuel_name.xlsx')
unique_fuels_df.to_excel(result_path, index = False)

result_path = os.path.join(folder_path, 'clean_egeda_sector_name.xlsx')
unique_sectors_df.to_excel(result_path, index = False)

### (Do it in excel) Compare the self-defined layout (在excel 製造對照表：自定義的欄位跟前面資整的欄位)
- create new files 
    - .\manuel_adjust\reference_table_fuel.xlsx 
    - .\manuel_adjust\reference_table_sector.xlsx
- copy the clean_egeda cols and the sd_layout, and then paste them to the excel files that you just created  
- align the name of fuels and sector, and mark the differences in other cols.

#### Check if there are still differences in the fuel name (ignore the item number)

In [ ]:
fuel_reference_table = pd.read_excel('./data/manuel_adjust/reference_table_fuel.xlsx')

In [ ]:
# drop the item number (把編號拿掉，並用迴圈逐步清除)
for i in range(0, 2, 1):
    fuel_reference_table['clean_egeda_fuel_name'] = fuel_reference_table['clean_egeda_fuel_name'].str.replace(r'^\d*_', '')
    fuel_reference_table['unique_the_end_of_fuels'] = fuel_reference_table['unique_the_end_of_fuels'].str.replace(r'^\d*_', '')

In [ ]:
# list the differences
fuel_reference_table['not_equal'] = fuel_reference_table['clean_egeda_fuel_name'] != fuel_reference_table['unique_the_end_of_fuels']

In [ ]:
diff_fuel = fuel_reference_table[fuel_reference_table['not_equal'] == True].sort_values(['how', 'unique_the_end_of_fuels'])
# diff_fuel.tail(10)

In [ ]:
# # format difference that should be revise in excel
# diff_fuel[(diff_fuel['clean_egeda_fuel_name'].notnull()) & (diff_fuel['unique_the_end_of_fuels'].notnull())]

#### Check if there are still differences in the sector name (ignore the item number)

In [ ]:
sector_reference_table = pd.read_excel('./data/manuel_adjust/reference_table_sector.xlsx')

In [ ]:
# drop the item number (把編號拿掉，並用迴圈逐步清除)
for i in range(0, 3, 1):
    sector_reference_table['clean_egeda_sector_name'] = sector_reference_table['clean_egeda_sector_name'].str.replace(r'^\d*_', '')
    sector_reference_table['unique_the_end_of_sectors'] = sector_reference_table['unique_the_end_of_sectors'].str.replace(r'^\d*_', '')

In [ ]:
# list the differences
sector_reference_table['not_equal'] = sector_reference_table['clean_egeda_sector_name'] != sector_reference_table['unique_the_end_of_sectors']

In [ ]:
diff_sector = sector_reference_table[sector_reference_table['not_equal'] == True].sort_values(['how', 'unique_the_end_of_sectors'])

In [ ]:
# # format difference that should be revise in excel
# diff_sector[(diff_sector['clean_egeda_sector_name'].notnull()) & (diff_sector['unique_the_end_of_sectors'].notnull())]

### (Do it in excel) Fix the format difference
- Ideally, the item names of cleaned EGEDA and self-defined layout should be the same.
- This section gives you a change to check and revise accidental difference.
- ```diff_fule```and ```diff_sector``` allow you to check the difference.
---
- "format difference that should be revise in excel" shows data format you should revise in excel
- Afetr revision, except for the item number, the data format should be the same.
- Save them as new files 
    - .\data\reference_table_fuel_revised.xlsx 
    - .\data\reference_table_sector_revised.xlsx 
    
---
FYI, the differences in data format
1. Replace & with
    - Wu: _
    - Mat: and
2. unnecessary commas
3. typos from original EGEDA dataset: It should be liquor instead of liqour.

**總結一下對照的發現**
- 除了新增欄位外，命名差異主要為 & 
    - 我用 _ 取代
    - Mat 用 and 取代
- 在 sector 中，有部分的資料中有多餘的逗號  
- 在 fuel 中，有一個拼錯的字：'liqour'，改成 'liquor'

# Connect the sd_layout with the EGEDA historical data (df)
1. Improt the revised reference table
2. "clean_egeda_fuel_name" and "fuels" have the same values but different data points. We use this property to connect the historical data (df) to sd_layout.
3. Create rows (new rows, minus) and aggregate rows (create new ones and discard old ones).
4. Merge the multiple cols (indicate different levels) to the EGEDA historical data (df).
---
**下一步**
- 接下來的作業就是將新的對照表、layout 引入
- 依照對照表，將清理後的 sector 跟 fuel 取代為我的版本
- 處理相加、相減、以及新增欄位，記得把相加後不需要的素材欄位刪掉
- 透過 merge 讓 layout 的多欄位正確對應資料框

## Import data

In [ ]:
fuel_mapping = pd.read_excel('./data/manuel_adjust/reference_table_fuel_revised.xlsx', usecols = [0, 1])
sector_mapping = pd.read_excel('./data/manuel_adjust/reference_table_sector_revised.xlsx', usecols = [0, 1])
# 這兩份資料均處理了多餘的逗號、拼錯字的部分

## Replace the data in "fuels" series with "unique_the_end_of_fuels" series (將 fuels 取代為自定義版本)

### Merge the "fuel_mapping" with "df" 

In [ ]:
# We don't need the multiple index (ecnomy, year)
df_no_year_econ_index = df.reset_index()

In [ ]:
# Notice that the values in "fuels" and "clean_egeda_fuel_name" are the same
# You can check with the following code
# test = df_fuel.loc[df_fuel['fuels'] != df_fuel['clean_egeda_fuel_name']]
# test

df_fuel = pd.merge(df_no_year_econ_index, fuel_mapping, how = 'left', left_on = 'fuels', right_on = 'clean_egeda_fuel_name')
# df_fuel

In [ ]:
# df_fuel[((df_fuel['fuels'] == '03_peat') | (df_fuel['fuels'] == '04_peat_products')) & ((df_fuel['value'] != 0))]

In [ ]:
# df_fuel[df_fuel['unique_the_end_of_fuels'] == '16_11_ammonia']

In [ ]:
# # example of same item name/ format but different item_number: unique_the_end_of_fuels
# df_fuel[df_fuel['fuels']=='01_05_lignite']

## Replace the fules series
1. create "replace_materials" series that is based on "unique_the_end_of_fuels" series with NaN filled by fuels
2. The content of "replace_materials" series ```df_fuel['replace_materials'] = df_fuel['unique_the_end_of_fuels'].fillna(df_fuel['fuels'])
```
    - "fuels"-null; "unique_the_end_of_fuels"-notnull → use "unique_the_end_of_fuels"
    - "fuels"-notnull; "unique_the_end_of_fuels"-notnull → use "unique_the_end_of_fuels" 
    - "fuels"-notnull; "unique_the_end_of_fuels"-null → use "fuels"

| fules    | replace | uni    |
| -------- | :-----: | :---   |
| Alice→   |  Alice  | nan    |
| nan      |  Bob    | ←Bob   |
| Charlie  |  David  | ←David |


3. Use "replace_materials" series to replace the original 'fuels' series
    - Although it's impossible to have nan "replace_materials" series, we still use  ```.where(df_fuel['replace_materials'].notnull()``` in the script
| fules  | replace |
| ------ | :------ |
| Alice  | Alice   |
| Bob    | ←Bob    |
| David  | ←David  |



In [ ]:
df_fuel['replace_materials'] = df_fuel['unique_the_end_of_fuels'].fillna(df_fuel['fuels'])
df_fuel['fuels'] = df_fuel['replace_materials'].where(df_fuel['replace_materials'].notnull(), df_fuel['fuels'])

In [ ]:
# # Check the result: If we succesfully replace the series, we should not filter out '01_03_lignite' (the item number is changed).
# # 如果上式成功將自定義的名稱取代 fuels，則這個篩選結果應該找不到資料，因為自訂的的這筆資料已經換編號了
# df_fuel[df_fuel['fuels']=='01_05_lignite']  

In [ ]:
# df_fuel[df_fuel['fuels']=='01_03_lignite']

In [ ]:
df_fuel.drop(['clean_egeda_fuel_name', 'unique_the_end_of_fuels', 'replace_materials'], axis= 1, inplace = True)

## Replace the data in "sectors" series with "unique_the_end_of_sectors" series (將 sectors 取代為我的版本)
- similar to the process of fules

In [ ]:
df_fuel_sector = pd.merge(df_fuel, sector_mapping, how = 'left', left_on = 'sectors', right_on = 'clean_egeda_sector_name')

In [ ]:
df_fuel_sector['replace_materials'] = df_fuel_sector['unique_the_end_of_sectors'].fillna(df_fuel_sector['sectors'])
df_fuel_sector['sectors'] = df_fuel_sector['replace_materials'].where(df_fuel_sector['replace_materials'].notnull(), df_fuel_sector['sectors'])

In [ ]:
df_fuel_sector.drop(['clean_egeda_sector_name', 'unique_the_end_of_sectors', 'replace_materials'], axis= 1, inplace = True)

# self_defined layout

## Import the data

In [ ]:
fuel_layout = pd.read_excel('./data/self_defined_layout/EBT_column_fuels.xlsx', sheet_name = 'fuel_layout_20230316', usecols = [0, 1])
sector_layout = pd.read_excel('./data/self_defined_layout/EBT_row_sectors.xlsx', sheet_name = 'sector_layout_20230315', usecols = [0, 1, 2, 3, 4])

## clean the data again 
- Just in case, the data should be clean before this.

In [ ]:
for i in fuel_layout.columns:
    fuel_layout[i] =  fuel_layout[i].str.replace(' ', '_', regex = False)\
                                       .str.replace('.', '_', regex = False)\
                                       .str.replace('/', '_', regex = False)\
                                       .str.replace('(', '', regex = False)\
                                       .str.replace(')', '', regex = False)\
                                       .str.replace('-', '', regex = False)\
                                       .str.replace(',', '', regex = False)\
                                       .str.replace('&', 'and', regex = False)\
                                       .str.replace('___', '_', regex = False)\
                                       .str.replace('__', '_', regex = False)\
                                       .str.replace(':', '', regex = False)\
                                       .str.replace('liqour', 'liquor', regex = False)\
                                       .str.rstrip('_')\
                                        .replace('\s+', ' ', regex = True)\
                                        .str.lower()

In [ ]:
for i in sector_layout.columns:
    sector_layout[i] =  sector_layout[i].str.replace(' ', '_', regex = False)\
                                       .str.replace('.', '_', regex = False)\
                                       .str.replace('/', '_', regex = False)\
                                       .str.replace('(', '', regex = False)\
                                       .str.replace(')', '', regex = False)\
                                       .str.replace('-', '', regex = False)\
                                       .str.replace(',', '', regex = False)\
                                       .str.replace('&', 'and', regex = False)\
                                       .str.replace('___', '_', regex = False)\
                                       .str.replace('__', '_', regex = False)\
                                       .str.replace(':', '', regex = False)\
                                       .str.replace('liqour', 'liquor', regex = False)\
                                       .str.rstrip('_')\
                                        .replace('\s+', ' ', regex = True)\
                                        .str.lower()

## Create key col for you to connect the multi cols and EGEDA data (創造介接的欄位)
- key column is the lowest level of our self-define layout
- you can get the lowest level in excel and python.
- I did both anyway.

### fuel

In [ ]:
# subfuels is the lowest level
# We create a new col base on it
fuel_layout['fuel_key_col'] = fuel_layout['subfuels']

for i in range(len(fuel_layout)):
    if fuel_layout.loc[i, 'subfuels'] == 'x':
        fuel_layout.loc[i, 'fuel_key_col'] = fuel_layout.loc[i, 'fuels'] 

In [ ]:
# print(len(fuel_layout['fuel_key_col'].unique()), len(fuel_layout)) # should be the same

### sector

In [ ]:
# sub4sectors is the lowest level
# We create a new col base on it

sector_layout['sector_key_col'] = sector_layout['sub4sectors']

for i in range(len(sector_layout)):
    if (sector_layout.loc[i, 'sub4sectors'] == 'x') & (sector_layout.loc[i, 'sub3sectors'] != 'x'):
        sector_layout.loc[i, 'sector_key_col'] = sector_layout.loc[i, 'sub3sectors']
    else:
        if (sector_layout.loc[i, 'sub3sectors'] == 'x') & (sector_layout.loc[i, 'sub2sectors'] != 'x'):
            sector_layout.loc[i, 'sector_key_col'] = sector_layout.loc[i, 'sub2sectors']
        else:
            if (sector_layout.loc[i, 'sub2sectors'] == 'x') & (sector_layout.loc[i, 'sub1sectors'] != 'x'):
                sector_layout.loc[i, 'sector_key_col'] = sector_layout.loc[i, 'sub1sectors']
            else:
                if (sector_layout.loc[i, 'sub1sectors'] == 'x') & (sector_layout.loc[i, 'sectors'] != 'x'):
                    sector_layout.loc[i, 'sector_key_col'] = sector_layout.loc[i, 'sectors']

In [ ]:
# print(len(sector_layout['sector_key_col'].unique()), len(sector_layout)) # should be the same

In [ ]:
# df_fuel_sector[((df_fuel_sector['fuels'] == '03_peat') | (df_fuel_sector['fuels'] == '04_peat_products')) & ((df_fuel_sector['value'] != 0))]

# Aggregation, Disaggregation, New rows (相加相減作業)
- Notice that we deal with fuel, concating and excluding the targeted rows. 
- Based on the result, we deal with sectors, concating and excluding the trageted rows.
- DO NOT deal with these two together first, then concating and excluding the trageted rows. It may cause failure of merging or missing data.
---
- BTW, df_fuel_sector is the latest, cleaned dataframe of the EGEDA historical data.



## Fuel

### Aggregation: 01_02_thermal_coal

In [ ]:
# df_fuel_sector[df_fuel_sector['fuels'] == '01_02_other_bituminous_coal']

In [ ]:
thermal_coal = df_fuel_sector[df_fuel_sector['fuels'].isin(['01_02_other_bituminous_coal', '01_03_subbituminous_coal', '01_04_anthracite', '03_peat', '04_peat_products'])]

In [ ]:
# thermal_coal[(thermal_coal['sectors'] == '19_04_ap_heat_plants') & (thermal_coal['economy'] == 'APEC') & (thermal_coal['year'] == 2020)]

In [ ]:
thermal_coal_g = thermal_coal.groupby(['economy', 'year', 'sectors'])['value'].sum().reset_index().assign(fuels = '01_02_thermal_coal')

### Aggregation: 04_03_other_hydrocarbons

In [ ]:
other_hydrocarbons = df_fuel_sector[df_fuel_sector['fuels'].isin(['06_03_refinery_feedstocks', '06_04_additives_oxygenates', '06_05_other_hydrocarbons'])]

In [ ]:
# other_hydrocarbons[(other_hydrocarbons['sectors'] == '19_04_ap_heat_plants') & (other_hydrocarbons['economy'] == 'APEC') & (other_hydrocarbons['year'] == 2020)]

In [ ]:
other_hydrocarbons_g = other_hydrocarbons.groupby(['economy', 'year', 'sectors'])['value'].sum().reset_index().assign(fuels = '04_03_other_hydrocarbons')

### Aggregation: 05_04_jet_fuel

In [ ]:
jet_fuel = df_fuel_sector[df_fuel_sector['fuels'].isin(['07_04_gasoline_type_jet_fuel', '07_05_kerosene_type_jet_fuel'])]

In [ ]:
# jet_fuel[(jet_fuel['sectors'] == '19_04_ap_heat_plants') & (jet_fuel['economy'] == 'APEC') & (jet_fuel['year'] == 2020)]

In [ ]:
jet_fuel_g = jet_fuel.groupby(['economy', 'year', 'sectors'])['value'].sum().reset_index().assign(fuels = '05_04_jet_fuel')

### Aggregation: 05_11_other_petroleum_products

In [ ]:
other_petroleum_products = df_fuel_sector[df_fuel_sector['fuels'].isin(['07_12_white_spirit_sbp', '07_13_lubricants', '07_14_bitumen',\
                                                                        '07_15_paraffin_waxes', '07_16_petroleum_coke', '07_17_other_products'])]

In [ ]:
# other_petroleum_products[(other_petroleum_products['sectors'] == '19_04_ap_heat_plants') & (other_petroleum_products['economy'] == 'APEC') & (other_petroleum_products['year'] == 2020)]

In [ ]:
other_petroleum_products_g = other_petroleum_products.groupby(['economy', 'year', 'sectors'])['value'].sum().reset_index().assign(fuels = '05_11_other_petroleum_products')

### Disaggregation: 10_02_other_solar (minus)
- minus with the .groupby method

In [ ]:
other_solar = df_fuel_sector[df_fuel_sector['fuels'].isin(['10_solar', '10_01_of_which_photovoltaics'])]

In [ ]:
other_solar_a = other_solar[other_solar['fuels'] == '10_solar']
other_solar_b = other_solar[other_solar['fuels'] == '10_01_of_which_photovoltaics']

In [ ]:
other_solar_b['value'] = other_solar_b['value']* -1

In [ ]:
other_solar = pd.concat([other_solar_a, other_solar_b], axis = 0)

In [ ]:
# other_solar[(other_solar['sectors'] == '01_production') & (other_solar['economy'] == '01_AUS') & (other_solar['year'] == 1980)]

In [ ]:
other_solar_g = other_solar.groupby(['economy', 'year', 'sectors'])['value'].sum().reset_index().assign(fuels = '10_02_other_solar')

### New-defined fuel: 14_10_hydrogen and 14_11_ammonia

In [ ]:
temp_for_new = other_solar_g.copy()
temp_for_new['value'] = 0
temp_for_new['fuels'] = 0
# temp_for_new

In [ ]:
hydrogen = temp_for_new.copy()
hydrogen['fuels'] = '14_10_hydrogen'

ammonia = temp_for_new.copy()
ammonia['fuels'] = '14_11_ammonia'

### Concate 
- exclude the data that are used to do the aggreation (排除用來相加的資料)
- include the data you just create (加入相加相減後的資料)

In [ ]:
# 排除用來相加的資料
# Remember to remove the peat and peat product because we decided to let thme back to the list. (20230317)
df_fuel_sector = df_fuel_sector[~df_fuel_sector['fuels'].isin(['01_02_other_bituminous_coal', '01_03_subbituminous_coal', '01_04_anthracite', \
                                                                    '06_03_refinery_feedstocks', '06_04_additives_oxygenates', '06_05_other_hydrocarbons',\
                                                                    '07_04_gasoline_type_jet_fuel', '07_05_kerosene_type_jet_fuel', '07_12_white_spirit_sbp', '07_13_lubricants',\
                                                                    '07_14_bitumen', '07_15_paraffin_waxes', '07_16_petroleum_coke', '07_17_other_products'])]

In [ ]:
# 加入相加相減後的資料
df_fuel_sector = pd.concat([df_fuel_sector,\
                                 thermal_coal_g, other_hydrocarbons_g, jet_fuel_g, other_petroleum_products_g, other_solar_g,\
                                 hydrogen, ammonia], axis = 0).reset_index(drop = True)

## Sector

### Aggregation: 09_01_electricity_plants

In [ ]:
ele_tf = df_fuel_sector[df_fuel_sector['sectors'].isin(['09_01_01_electricity_plants', '09_02_01_electricity_plants'])]
# ele_tf.tail(10)

In [ ]:
ele_tf_g = ele_tf.groupby(['economy', 'year', 'fuels'])['value'].sum().reset_index().assign(sectors = '09_01_electricity_plants')
# ele_tf_g

In [ ]:
# ele_tf_g[ele_tf_g['economy'] == '25_OCE']

###  Aggregation: 09_02_chp_plants

In [ ]:
chp_tf = df_fuel_sector[df_fuel_sector['sectors'].isin(['09_01_02_chp_plants', '09_02_02_chp_plants'])]
# chp_tf.tail(10)

In [ ]:
chp_tf_g = chp_tf.groupby(['economy', 'year', 'fuels'])['value'].sum().reset_index().assign(sectors = '09_02_chp_plants')
# chp_tf_g

In [ ]:
# chp_tf_g[ele_tf_g['economy'] == '25_OCE']

### Aggregation: 09_03_heat_plants

In [ ]:
heat_tf = df_fuel_sector[df_fuel_sector['sectors'].isin(['09_01_03_heat_plants', '09_02_03_heat_plants'])]
# heat_tf.tail(10)

In [ ]:
# heat_tf[heat_tf['economy'] =='APEC'].tail(10)

In [ ]:
heat_tf_g = heat_tf.groupby(['economy', 'year', 'fuels'])['value'].sum().reset_index().assign(sectors = '09_03_heat_plants')

In [ ]:
# heat_tf_g[heat_tf_g['economy'] == 'APEC']

### Aggregation: 16_01_building

In [ ]:
bldg = df_fuel_sector[df_fuel_sector['sectors'].isin(['16_01_01_commercial_and_public_services', '16_01_02_residential'])] # notice that the item numbers are replaced by self-defined version.
# bldg.tail(10)

In [ ]:
bldg_g = bldg.groupby(['economy', 'year', 'fuels'])['value'].sum().reset_index().assign(sectors = '16_01_building')
# bldg_g

In [ ]:
# bldg_g[bldg_g['economy'] == '25_OCE']

### Aggregation: 16_02_agriculture_and_fishing

In [ ]:
agfi = df_fuel_sector[df_fuel_sector['sectors'].isin(['16_02_01_agriculture', '16_02_02_fishing'])] # notice that the item numbers are replaced by self-defined version.
# agfi.tail(10)

In [ ]:
agfi_g = agfi.groupby(['economy', 'year', 'fuels'])['value'].sum().reset_index().assign(sectors = '16_02_agriculture_and_fishing')
# bldg_g

In [ ]:
# agfi_g[agfi_g['economy'] == '25_OCE']

### Aggregation: 18_01_electricity_plants

In [ ]:
ele_gwh = df_fuel_sector[df_fuel_sector['sectors'].isin(['18_01_map_electricity_plants', '18_03_ap_electricity_plants'])] 
# ele_gwh.tail(10)

In [ ]:
ele_gwh_g = ele_gwh.groupby(['economy', 'year', 'fuels'])['value'].sum().reset_index().assign(sectors = '18_01_electricity_plants')

In [ ]:
# ele_gwh_g[ele_gwh_g['economy'] == '25_OCE']

In [ ]:
# ele_gwh_g[(ele_gwh_g['economy'] == '01_AUS') & (ele_gwh_g['fuels'] == '')]

### Aggregation: 18_02_chp_plants
- electricity

In [ ]:
chp_gwh = df_fuel_sector[df_fuel_sector['sectors'].isin(['18_02_map_chp_plants', '18_04_ap_chp_plants'])] 
# chp_gwh.tail(10)

In [ ]:
chp_gwh_g = chp_gwh.groupby(['economy', 'year', 'fuels'])['value'].sum().reset_index().assign(sectors = '18_02_chp_plants')

In [ ]:
# chp_gwh_g[chp_gwh_g['economy'] == '25_OCE']

### Aggregation: 19_01_chp plants
- heat

In [ ]:
chp_pj = df_fuel_sector[df_fuel_sector['sectors'].isin(['19_01_map_chp_plants', '19_03_ap_chp_plants'])] 
# chp_pj.tail(10)

In [ ]:
chp_pj_g = chp_pj.groupby(['economy', 'year', 'fuels'])['value'].sum().reset_index().assign(sectors = '19_01_chp plants')

In [ ]:
chp_pj_g[chp_pj_g['economy'] == '25_OCE']

### Aggregation: 19_02_heat_plants

In [ ]:
heat_pj = df_fuel_sector[df_fuel_sector['sectors'].isin(['19_02_map_heat_plants', '19_04_ap_heat_plants'])] 
# heat_pj.tail(10)

In [ ]:
# heat_pj[heat_pj['economy'] =='APEC'].tail(10)

In [ ]:
heat_pj_g = heat_pj.groupby(['economy', 'year', 'fuels'])['value'].sum().reset_index().assign(sectors = '19_02_heat_plants')

In [ ]:
# heat_pj_g[heat_pj_g['economy'] == 'APEC']

### concate for sectors
- exclude the data that are used to do the aggreation (排除用來相加的資料)
- include the data you just create (加入相加相減後的資料)

In [ ]:
df_fuel_sector = df_fuel_sector[~df_fuel_sector['sectors'].isin(['09_01_main_activity_producer', '09_02_autoproducers',\
                                                                                '09_01_01_electricity_plants', '09_02_01_electricity_plants',\
                                                                                '09_01_02_chp_plants', '09_02_02_chp_plants',\
                                                                                '09_01_03_heat_plants', '09_02_03_heat_plants',\
                                                                                '18_01_map_electricity_plants', '18_03_ap_electricity_plants',\
                                                                                '18_02_map_chp_plants', '18_04_ap_chp_plants'])]

In [ ]:
# 加入相加相減後的資料
df_fuel_sector_temp = pd.concat([df_fuel_sector,\
                                 ele_tf_g, chp_tf_g, heat_tf_g,\
                                 bldg_g, agfi_g, \
                                 ele_gwh_g, chp_gwh_g, \
                                 chp_pj_g, heat_pj_g], axis = 0).reset_index(drop = True)

# Merge the layout and the EGEDA data

## Merge the layout: sector, fuel, econ, year

In [ ]:
sector_fuel_layout = pd.merge(sector_layout, fuel_layout, how = 'cross')  

In [ ]:
economy_df = pd.DataFrame(df_fuel_sector_temp['economy'].unique(), columns=['economy'])
year_df = pd.DataFrame(df_fuel_sector_temp['year'].unique(), columns=['year'])

In [ ]:
df_econ_year = pd.merge(economy_df, year_df, how = 'cross')

In [ ]:
sector_fuel_econ_year_layout =  pd.merge(df_econ_year, sector_fuel_layout, how = 'cross')

## Merge layout and EGEDA data

In [ ]:
merged_df = pd.merge(df_fuel_sector_temp, sector_fuel_econ_year_layout, how = 'right', \
                     left_on = ['economy', 'year', 'fuels', 'sectors'], \
                     right_on = ['economy', 'year', 'fuel_key_col', 'sector_key_col'])  

In [ ]:
# merged_df['fuel_key_col'].unique()

# Drop the useless col

In [ ]:
merged_df_clean = merged_df.drop(['fuels_x', 'sectors_x', 'sector_key_col', 'fuel_key_col'], axis = 1)

In [ ]:
merged_df_clean = merged_df_clean.rename(columns={'sectors_y': 'sectors', 'fuels_y': 'fuels'})

In [ ]:
merged_df_clean = merged_df_clean.reindex(columns=['economy', 'year', 'sectors', 'sub1sectors', 'sub2sectors', 'sub3sectors', 'sub4sectors', 'fuels', 'subfuels', 'value'])

In [ ]:
# merged_df_clean

In [ ]:
# merged_df_clean[(merged_df_clean['economy'] == '01_AUS') & (merged_df_clean['sub1sectors'] == '09_01_electricity_plants')\
#                 & (merged_df_clean['subfuels'] == '01_02_thermal_coal') ]

# Export the merged result (將結果輸出)

## pivot the data first to save time
- You have to fillna with some numerical number first, otherwise the row with np.nan will be ignore by ```pivot_table``` (補數字，否則 pivot 會把都是 np.NAN 的欄位給忽略掉)
- ```value_not_in_the_range``` is assigned to a value that is differnt from the current dataframe. (series) 
- We will replace them with np.nan after ```pivot_table```. (value_not_in_the_range 目的是之後把數值取代回 np.nan)
- BTW, replacing with string may cause problem in ```pivot_table```

In [ ]:
value_not_in_the_range = merged_df_clean['value'].min() - 1
merged_df_clean = merged_df_clean.fillna(value_not_in_the_range)
# merged_df_clean

In [ ]:
merged_df_clean_wide = merged_df_clean.pivot_table(index = ['economy', 'sectors', 'sub1sectors', 'sub2sectors', 'sub3sectors', 'sub4sectors', 'fuels', 'subfuels'],\
                                       columns = 'year', values = 'value').reset_index(drop = False)

In [ ]:
# merged_df_clean_wide

In [ ]:
# merged_df_clean_wide['sub3sectors'].unique()

In [ ]:
# merged_df_clean_wide[merged_df_clean_wide['sub3sectors'] == '18_xx_01_10_otherrenewable']

In [ ]:
# merged_df_clean_wide[merged_df_clean_wide['subfuels'] == '14_11_ammonia']

## merge the scenarios

In [ ]:
scen = pd.read_excel('./data/scenario_list.xlsx')

In [ ]:
merged_df_clean_wide = pd.merge(scen, merged_df_clean_wide, how = 'cross')

## Extend the projected years (2021 to 2070)

In [ ]:
projected_years = list(range(2021, 2070+1, 1))
# projected_years

In [ ]:
for i in projected_years:
    merged_df_clean_wide[i] = np.nan

## Replace the ```value_not_in_the_range``` with ```np.nan```
- In principle, rows without historical data will be assign np.nan rather than 0.
---
- Notice that we set 0 for hydrogen and ammonia before concating.
- They became ```np.nan``` during the process.
- To do the pivot properly, ther became value_not_in_the_range.
- And now they are going to become np.nan again.

In [ ]:
merged_df_clean_wide.replace(value_not_in_the_range, np.nan, inplace = True)

# Export the data

## Entire data

In [ ]:
from datetime import datetime
date_today = datetime.now().strftime('%Y%m%d')
# date_today

In [ ]:
import os
folder_path = './result'
os.makedirs(folder_path, exist_ok=True)

file_name = 'output_merged_df_clean_wide_' + date_today +'.csv'
result_path = os.path.join(folder_path, file_name)
merged_df_clean_wide.to_csv(result_path, index = False)

## Choose one economy

In [ ]:
merged_df_clean_wide_aus = merged_df_clean_wide[merged_df_clean_wide['economy'] == '01_AUS']
# merged_df_clean_wide_aus

In [ ]:
import os
folder_path = './result'
os.makedirs(folder_path, exist_ok=True)

file_name = 'output_merged_df_clean_wide_aus_' + date_today +'.csv'
result_path = os.path.join(folder_path, file_name)
merged_df_clean_wide_aus.to_csv(result_path, index = False)

## Choose one economy and one scenario

In [ ]:
merged_df_clean_wide_aus_net_zero = merged_df_clean_wide_aus[merged_df_clean_wide_aus['scenarios'] == 'net-zero']

In [ ]:
import os
folder_path = './result'
os.makedirs(folder_path, exist_ok=True)

file_name = 'output_merged_df_clean_wide_aus_net_zero_' + date_today +'.csv'
result_path = os.path.join(folder_path, file_name)
merged_df_clean_wide_aus_net_zero.to_csv(result_path, index = False)